In [61]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
print("Setup Complete")
from pandas_profiling import ProfileReport
from pandas_profiling.utils.cache import cache_file
    

def classificateWithNumbersThreeTarget(value):
    target=0
    if(value>=0.0015):
        target=1
    elif(value<=-0.0015):
        target=-1
    else:
        target=0
    return target




def correlation_heatmap(df):
    correlations = df.corr()

    fig, ax = plt.subplots(figsize=(40,40))
    sns.heatmap(correlations, vmax=1.0, center=0, fmt='.2f',
                square=True, linewidths=.1, annot=True, cbar_kws={"shrink": .7})
    sns.set(font_scale=2)
    plt.show();

def aboveValue(openList,inspectList,df,name):
    results=[]
    for i in range(len(inspectList)):
        if (openList[i]>inspectList[i]):
            results.append(1)
        else:
            results.append(0)
    df[name]=results
    
data1 = pd.read_csv('/home/balazs/Desktop/adathalmazok/btc_2012-2021/btc_split_data_part_06', sep=",", header=None)
data1.columns = ["time", "open", "high", "low","close","volume",'drop1','drop2']

data2 = pd.read_csv('/home/balazs/Desktop/adathalmazok/btc_2012-2021/btc_split_data_part_07', sep=",", header=None)
data2.columns = ["time", "open", "high", "low","close","volume",'drop1','drop2']

data3 = pd.read_csv('/home/balazs/Desktop/adathalmazok/btc_2012-2021/btc_split_data_part_08', sep=",", header=None)
data3.columns = ["time", "open", "high", "low","close","volume",'drop1','drop2']

data4 = pd.read_csv('/home/balazs/Desktop/adathalmazok/btc_2012-2021/btc_split_data_part_09', sep=",", header=None)
data4.columns = ["time", "open", "high", "low","close","volume"]

data= pd.concat([data1,data2,data3,data4], axis=0)
#data

#data=data4

data_test = pd.read_csv('/home/balazs/Desktop/adathalmazok/1weekMinedData.txt', sep=";", header=None)
data_test.columns = ["time", "open", "high", "low","close","volume",'temp1','temp2']

#data=data[['time','open','high','low','close','volume']]
#data=data.dropna()
#data=data[4000:]
    

Setup Complete


In [62]:
%%time
#btc_data = pd.read_csv(r"/home/balazs/Desktop/szakdoga_info/btc_data.csv")
#data = btc_data.dropna()
data['MA5']=data['open'].rolling(window =5).mean()
data['EMA5']=data['open'].ewm(span=5,min_periods=0,adjust=False,ignore_na=False).mean()
data['CMA']=data.open.expanding().mean()
data['open_percentage_change']=data.open.pct_change()
data['close_percentage_change']=data.close.pct_change()
data['high_percentage_change']=data.high.pct_change()
data['low_percentage_change']=data.low.pct_change()
data['volume_percentage_change']=data.volume.pct_change()
lista=[]
for i in range(len(data.open_percentage_change.values)-5):
        value=data.open_percentage_change.values[i+1]+data.open_percentage_change.values[i+2]+data.open_percentage_change.values[i+3]+data.open_percentage_change.values[i+4]+data.open_percentage_change.values[i+5]
        lista.append(value)
for i in range(5):
    lista.append(0)
data['profit']=lista
buy_sell_list=[classificateWithNumbersThreeTarget(elem) for elem in data.profit]
data['decision']=buy_sell_list
data=data.dropna()
data.drop(data.tail(5).index,inplace=True) # drop last n rows
aboveValue(data.open.values,data.MA5.values,data,'aboveMA5')
aboveValue(data.open.values,data.EMA5.values,data,'aboveEMA5')
aboveValue(data.open.values,data.CMA.values,data,'aboveCMA')

CPU times: user 44.5 s, sys: 672 ms, total: 45.1 s
Wall time: 45.2 s


In [63]:
X=data[['close_percentage_change','high_percentage_change','low_percentage_change']]
y=data[['decision']]

In [64]:
from imblearn.combine import SMOTETomek
smt = SMOTETomek(random_state=42)
X, y = smt.fit_resample(X, y)

In [65]:
y.value_counts()

decision
-1          800074
 1          799442
 0          761326
dtype: int64

In [66]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

In [67]:
from sklearn.neighbors import KNeighborsClassifier

In [22]:
model_kn = KNeighborsClassifier(n_neighbors=9, leaf_size=20)

In [23]:
model_kn.fit(X_train, y_train)

KNeighborsClassifier(leaf_size=20, n_neighbors=9)

In [24]:
pred_kn = model_kn.predict(X_test)

In [25]:
from sklearn.metrics import classification_report
classes = ['-1','0','1']
print(classification_report(y_test, pred_kn, target_names = classes, zero_division = 0))

              precision    recall  f1-score   support

          -1       0.64      0.76      0.69     61001
           0       0.64      0.49      0.55     57747
           1       0.67      0.70      0.69     60708

    accuracy                           0.65    179456
   macro avg       0.65      0.65      0.64    179456
weighted avg       0.65      0.65      0.65    179456



In [32]:
from sklearn.model_selection import GridSearchCV
estimator_KNN = KNeighborsClassifier(algorithm='auto')

                   


In [44]:
knn = KNeighborsClassifier()
from sklearn.model_selection import GridSearchCV
parameters_KNN = {
    'n_neighbors': (1,5,6,7,8,9,10),
    'leaf_size': (20,30,40,50),
    'p': (1,2)}
  
grid = GridSearchCV(knn, parameters_KNN, scoring='f1', return_train_score=False,verbose=1)
  
# fitting the model for grid search
grid_search=grid.fit(X_train, y_train)

Fitting 5 folds for each of 56 candidates, totalling 280 fits


In [45]:
print(grid_search.best_params_)

{'leaf_size': 20, 'n_neighbors': 1, 'p': 1}


In [68]:
model_kn = KNeighborsClassifier(n_neighbors=1, leaf_size=20,p=1)

In [69]:
model_kn.fit(X_train, y_train)

KNeighborsClassifier(leaf_size=20, n_neighbors=1, p=1)

In [70]:
pred_kn = model_kn.predict(X_test)

In [71]:
from sklearn.metrics import classification_report
classes = ['-1','0','1']
print(classification_report(y_test, pred_kn, target_names = classes, zero_division = 0))

              precision    recall  f1-score   support

          -1       0.78      0.87      0.82    240231
           0       0.79      0.62      0.69    227961
           1       0.78      0.87      0.82    240061

    accuracy                           0.79    708253
   macro avg       0.79      0.78      0.78    708253
weighted avg       0.79      0.79      0.78    708253



In [72]:
import joblib
joblib.dump(model_kn,'knear.joblib')

['knear.joblib']